In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.9 MB/s eta 0:00:00


In [20]:
import os
import sys
import numpy as np

from google.colab import drive

drive.mount('/content/drive')
project_path = './drive/MyDrive/Colab Notebooks/GPT_community/'
sys.path.append(project_path)

from our_new_community.philosopher import Philosopher


def load_prompts(project_path, file_path):
    """Get a list of starting prompts from a text file."""

    prompts_file = os.path.join(project_path, file_path)
    with open(prompts_file) as file:
        prompts = [line.rstrip() for line in file]

    return prompts


def update_prompt(response, prompt_list, convo):
    """Update prompt and record of full conversation with response, and
    prune response of old content to keep below max context length.

    Args:
        response (str): The most recent response from a model
        prompt_list (list): Each individual response currently in the prompt
        convo (list): Every single response including original prompt

    Returns:
        str: New prompt, including most recent response
        list: Updated prompt list
        list: Updated record of full conversation
    """

    convo.append(response)
    prompt_list.append(response)
    prompt = ' '.join(prompt_list)

    while len(prompt) > max_len:
        prompt_list = prompt_list[1:]
        prompt = ' '.join(prompt_list)

    return prompt, prompt_list, convo


prompts_path = 'data/brighton_philosophy_prompts.txt'
prompts = load_prompts(project_path, prompts_path)
# prompts

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
kant_config = {'name':'Kant',
               'model_name':'gpt2',
               'model_type':'GTP2'}
hume_config = {'name':'Hume',
               'model_name':'gpt2',
               'model_type':'GTP2'}

kant = Philosopher(kant_config)
hume = Philosopher(hume_config)

philosophers = {1:kant, 2:hume}
max_len = max([value.context_len for key, value in philosophers.items()])

In [12]:
prompter_name = 'Socrates'

# Get initial prompt from questions list
prompt = f"{prompter_name}: " + np.random.choice(prompts)
print(prompt)

Socrates: Lies and truth: What is all the fuss about?


In [11]:
steps=2
convo = [prompt]
prompt_list = [prompt]

# Get initial response from philosopher 1
response = philosophers[1].get_response(prompt)
prompt, prompt_list, convo = update_prompt(response, prompt_list, convo)

for step in range(steps):
    # Get response from philosopher 2
    response = philosophers[2].get_response(prompt)
    prompt, prompt_list, convo = update_prompt(response, prompt_list, convo)

    # Update philosopher 1 weights
    philosophers[1].update_weights(response)

    # Get response from philosopher 1
    response = philosophers[1].get_response(prompt)
    prompt, prompt_list, convo = update_prompt(response, prompt_list, convo)

    # Update philosopher 2 weights
    philosophers[2].update_weights(response)